In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import librosa

# Base folder with 300_P, 301_P, ...
base_path = Path(r"C:\Users\DELL\Desktop\Conversational-Health-Analytics-\Dataset\extracted_folders")

# Get participant folder names like ['300_P', '301_P', ...]
participant_folders = sorted(
    [f for f in base_path.iterdir() if f.is_dir() and f.name.endswith("_P")]
)

print("Number of participants:", len(participant_folders))
print("Example folders:", [f.name for f in participant_folders[:5]])

Number of participants: 189
Example folders: ['300_P', '301_P', '302_P', '303_P', '304_P']


In [2]:
from pathlib import Path
import pandas as pd

pid = 300
folder = base_path / f"{pid}_P"
transcript_path = folder / f"{pid}_TRANSCRIPT.csv"

df_t = pd.read_csv(transcript_path, sep="\t")  # IMPORTANT
print("Columns in transcript file:", df_t.columns.tolist())
df_t.head()

Columns in transcript file: ['start_time', 'stop_time', 'speaker', 'value']


,start_time,stop_time,speaker,value
0,36.588,39.668,Ellie,hi i'm ellie thanks for coming in today
1,39.888,43.378,Ellie,i was created to talk to people in a safe and ...
2,43.728,48.498,Ellie,think of me as a friend i don't judge i can't ...
3,49.188,52.388,Ellie,i'm here to learn about people and would love ...
4,52.658,58.958,Ellie,i'll ask a few questions to get us started and...


In [3]:
import librosa
import numpy as np
import pandas as pd

def load_audio_and_clean(pid, sr_target=None):
    """
    Loads original audio AND cleaned (no Ellie) audio.
    Prints both durations.
    Returns: y_full, y_clean, sr
    """
    folder = base_path / f"{pid}_P"
    audio_path = folder / f"{pid}_AUDIO.wav"
    transcript_path = folder / f"{pid}_TRANSCRIPT.csv"

    # Load original audio
    y_full, sr = librosa.load(audio_path, sr=sr_target)
    full_duration = len(y_full) / sr

    # Load transcript (tab-separated)
    df_t = pd.read_csv(transcript_path, sep="\t")
    
    # Ensure numeric times
    df_t["start_time"] = pd.to_numeric(df_t["start_time"], errors="coerce")
    df_t["stop_time"]  = pd.to_numeric(df_t["stop_time"],  errors="coerce")

    keep_segments = []

    for _, row in df_t.iterrows():
        speaker = str(row["speaker"]).lower()

        # Keep only participant speech
        if "ellie" in speaker:
            continue

        start_sec = float(row["start_time"])
        end_sec   = float(row["stop_time"])
        if np.isnan(start_sec) or np.isnan(end_sec):
            continue

        start_idx = int(start_sec * sr)
        end_idx   = int(end_sec * sr)

        # Clip to audio boundaries
        start_idx = max(0, min(start_idx, len(y_full)))
        end_idx   = max(0, min(end_idx, len(y_full)))

        if end_idx > start_idx:
            keep_segments.append(y_full[start_idx:end_idx])

    # Concatenate kept segments
    if keep_segments:
        y_clean = np.concatenate(keep_segments)
    else:
        y_clean = y_full  # fallback

    clean_duration = len(y_clean) / sr

    # Print both lengths
    print(f"Participant {pid}:")
    print(f" - Original audio length: {full_duration:.2f} seconds")
    print(f" - Cleaned audio length (no Ellie): {clean_duration:.2f} seconds")

    return y_full, y_clean, sr




### Summarize Feature Matrix.

In [4]:
import numpy as np
import pandas as pd

def summarize_feature_matrix(mat, prefix):
    """
    mat: 2D numpy array (n_features x n_frames)
    prefix: string for column names, e.g. 'mfcc', 'delta', 'delta2'
    
    Returns: pandas.Series with stats for each row:
             {prefix}_{i}_mean, _std, _min, _max
    """
    stats = {}
    
    n_features = mat.shape[0]
    for i in range(n_features):
        row = mat[i, :]
        row = row[np.isfinite(row)]  # just in case

        if row.size == 0:
            mean = std = rmin = rmax = np.nan
        else:
            mean = row.mean()
            std  = row.std()
            rmin = row.min()
            rmax = row.max()
        
        stats[f"{prefix}_{i+1}_mean"] = mean
        stats[f"{prefix}_{i+1}_std"]  = std
        stats[f"{prefix}_{i+1}_min"]  = rmin
        stats[f"{prefix}_{i+1}_max"]  = rmax

    return pd.Series(stats)

In [6]:
### LogMel Spectogram Features.

In [7]:
import librosa
import numpy as np
import pandas as pd

def extract_logmel_family_features(y, sr, n_mels=64, prefix="logmel"):
    """
    Extract Log-Mel Spectrogram + Delta + Delta-Delta
    Then summarize each into statistical features.

    Returns:
        pandas.Series (one row of features)
    """

    feats = {}

    # -------------------------------------------------
    # 1. Mel-Spectrogram (power)
    # -------------------------------------------------
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_mels=n_mels,
        power=2.0     # energy spectrogram
    )

    # Convert to log-mel
    logmel = librosa.power_to_db(mel, ref=np.max)

    mel_stats = summarize_feature_matrix(logmel, prefix=f"{prefix}")
    feats.update(mel_stats.to_dict())

    # -------------------------------------------------
    # 2. Delta (1st-order derivative)
    # -------------------------------------------------
    mel_delta = librosa.feature.delta(logmel)
    mel_delta_stats = summarize_feature_matrix(mel_delta, prefix=f"{prefix}_delta")
    feats.update(mel_delta_stats.to_dict())

    # -------------------------------------------------
    # 3. Delta-Delta (2nd-order derivative)
    # -------------------------------------------------
    mel_delta2 = librosa.feature.delta(logmel, order=2)
    mel_delta2_stats = summarize_feature_matrix(mel_delta2, prefix=f"{prefix}_delta2")
    feats.update(mel_delta2_stats.to_dict())

    return pd.Series(feats)


In [8]:
pid = 300
y_full, y_clean, sr = load_audio_and_clean(pid)

logmel_series = extract_logmel_family_features(y_clean, sr)
logmel_series["Participant_ID"] = pid

df_logmel = logmel_series.to_frame().T

print("Log-Mel DF shape:", df_logmel.shape)
display(df_logmel)


Participant 300:
 - Original audio length: 648.50 seconds
 - Cleaned audio length (no Ellie): 155.76 seconds
Log-Mel DF shape: (1, 769)


,logmel_1_mean,logmel_1_std,logmel_1_min,logmel_1_max,logmel_2_mean,logmel_2_std,logmel_2_min,logmel_2_max,logmel_3_mean,logmel_3_std,...,logmel_delta2_62_max,logmel_delta2_63_mean,logmel_delta2_63_std,logmel_delta2_63_min,logmel_delta2_63_max,logmel_delta2_64_mean,logmel_delta2_64_std,logmel_delta2_64_min,logmel_delta2_64_max,Participant_ID
0,-39.851742,5.740917,-50.290749,-10.951934,-36.666473,9.424625,-59.953186,-1.421984,-32.704754,10.618219,...,2.475754,0.000095,0.306427,-2.576145,2.543875,0.000019,0.286238,-2.665952,2.558672,300.0


In [10]:
from pathlib import Path
import pandas as pd

# ---- 1. Get all participant IDs ----
participant_ids = sorted([
    int(f.name.split("_")[0])
    for f in base_path.iterdir()
    if f.is_dir() and f.name.endswith("_P")
])

print("Total participants detected:", len(participant_ids))
print("First 10 participants:", participant_ids[:10])

Total participants detected: 189
First 10 participants: [300, 301, 302, 303, 304, 305, 306, 307, 308, 309]


In [12]:
# ---- 2. Prepare lists for each DF ----
logmel_rows     = []
# ---- 3. Loop over participants ----
for pid in participant_ids:
    print(f"\nProcessing Participant: {pid}")

    # Load full + cleaned audio
    y_full, y_clean, sr = load_audio_and_clean(pid)

    # 1) MFCC Family
    s_logmel = extract_logmel_family_features(y_clean, sr)
    s_logmel["Participant_ID"] = pid
    logmel_rows.append(s_logmel)


Processing Participant: 300
Participant 300:
 - Original audio length: 648.50 seconds
 - Cleaned audio length (no Ellie): 155.76 seconds

Processing Participant: 301
Participant 301:
 - Original audio length: 823.90 seconds
 - Cleaned audio length (no Ellie): 475.44 seconds

Processing Participant: 302
Participant 302:
 - Original audio length: 758.80 seconds
 - Cleaned audio length (no Ellie): 208.93 seconds

Processing Participant: 303
Participant 303:
 - Original audio length: 985.30 seconds
 - Cleaned audio length (no Ellie): 642.93 seconds

Processing Participant: 304
Participant 304:
 - Original audio length: 792.60 seconds
 - Cleaned audio length (no Ellie): 362.60 seconds

Processing Participant: 305
Participant 305:
 - Original audio length: 1704.00 seconds
 - Cleaned audio length (no Ellie): 1118.49 seconds

Processing Participant: 306
Participant 306:
 - Original audio length: 858.10 seconds
 - Cleaned audio length (no Ellie): 509.37 seconds

Processing Participant: 307
Par

In [13]:
import pandas as pd
from pathlib import Path

root = Path(r"C:\Users\DELL\Desktop\Conversational-Health-Analytics-")
dataset_folder = root / "Dataset"

DF_TRAIN = pd.read_csv(dataset_folder / "train_split_Depression_AVEC2017.csv")
DF_DEV   = pd.read_csv(dataset_folder / "dev_split_Depression_AVEC2017.csv")
DF_TEST  = pd.read_csv(dataset_folder / "full_test_split.csv")

In [14]:
df_logmel_all = pd.DataFrame(logmel_rows)

In [21]:
# Keep only needed label columns
label_cols = ["Participant_ID", "PHQ8_Score", "PHQ8_Binary", "Gender"]

df_train_lbl = DF_TRAIN[label_cols]
df_dev_lbl   = DF_DEV[label_cols]
df_test_lbl  = DF_TEST[label_cols]

df_labels = (
    pd.concat([df_train_lbl, df_dev_lbl, df_test_lbl], axis=0)
      .drop_duplicates(subset="Participant_ID")
      .reset_index(drop=True)
)

# Merge MFCC features with labels
logmel_with_labels = df_logmel_all.merge(df_labels, on="Participant_ID", how="inner")
print("MFCC+Labels shape:", logmel_with_labels.shape)
logmel_with_labels.head(2)

MFCC+Labels shape: (189, 772)


,logmel_1_mean,logmel_1_std,logmel_1_min,logmel_1_max,logmel_2_mean,logmel_2_std,logmel_2_min,logmel_2_max,logmel_3_mean,logmel_3_std,logmel_3_min,logmel_3_max,logmel_4_mean,logmel_4_std,logmel_4_min,logmel_4_max,logmel_5_mean,logmel_5_std,logmel_5_min,logmel_5_max,logmel_6_mean,logmel_6_std,logmel_6_min,logmel_6_max,logmel_7_mean,logmel_7_std,logmel_7_min,logmel_7_max,logmel_8_mean,logmel_8_std,logmel_8_min,logmel_8_max,logmel_9_mean,logmel_9_std,logmel_9_min,logmel_9_max,logmel_10_mean,logmel_10_std,logmel_10_min,logmel_10_max,logmel_11_mean,logmel_11_std,logmel_11_min,logmel_11_max,logmel_12_mean,logmel_12_std,logmel_12_min,logmel_12_max,logmel_13_mean,logmel_13_std,logmel_13_min,logmel_13_max,logmel_14_mean,logmel_14_std,logmel_14_min,logmel_14_max,logmel_15_mean,logmel_15_std,logmel_15_min,logmel_15_max,logmel_16_mean,logmel_16_std,logmel_16_min,logmel_16_max,logmel_17_mean,logmel_17_std,logmel_17_min,logmel_17_max,logmel_18_mean,logmel_18_std,logmel_18_min,logmel_18_max,logmel_19_mean,logmel_19_std,logmel_19_min,logmel_19_max,logmel_20_mean,logmel_20_std,logmel_20_min,logmel_20_max,logmel_21_mean,logmel_21_std,logmel_21_min,logmel_21_max,logmel_22_mean,logmel_22_std,logmel_22_min,logmel_22_max,logmel_23_mean,logmel_23_std,logmel_23_min,logmel_23_max,logmel_24_mean,logmel_24_std,logmel_24_min,logmel_24_max,logmel_25_mean,logmel_25_std,logmel_25_min,logmel_25_max,logmel_26_mean,logmel_26_std,logmel_26_min,logmel_26_max,logmel_27_mean,logmel_27_std,logmel_27_min,logmel_27_max,logmel_28_mean,logmel_28_std,logmel_28_min,logmel_28_max,logmel_29_mean,logmel_29_std,logmel_29_min,logmel_29_max,logmel_30_mean,logmel_30_std,logmel_30_min,logmel_30_max,logmel_31_mean,logmel_31_std,logmel_31_min,logmel_31_max,logmel_32_mean,logmel_32_std,logmel_32_min,logmel_32_max,logmel_33_mean,logmel_33_std,logmel_33_min,logmel_33_max,logmel_34_mean,logmel_34_std,logmel_34_min,logmel_34_max,logmel_35_mean,logmel_35_std,logmel_35_min,logmel_35_max,logmel_36_mean,logmel_36_std,logmel_36_min,logmel_36_max,logmel_37_mean,logmel_37_std,logmel_37_min,logmel_37_max,logmel_38_mean,logmel_38_std,logmel_38_min,logmel_38_max,logmel_39_mean,logmel_39_std,logmel_39_min,logmel_39_max,logmel_40_mean,logmel_40_std,logmel_40_min,logmel_40_max,logmel_41_mean,logmel_41_std,logmel_41_min,logmel_41_max,logmel_42_mean,logmel_42_std,logmel_42_min,logmel_42_max,logmel_43_mean,logmel_43_std,logmel_43_min,logmel_43_max,logmel_44_mean,logmel_44_std,logmel_44_min,logmel_44_max,logmel_45_mean,logmel_45_std,logmel_45_min,logmel_45_max,logmel_46_mean,logmel_46_std,logmel_46_min,logmel_46_max,logmel_47_mean,logmel_47_std,logmel_47_min,logmel_47_max,logmel_48_mean,logmel_48_std,logmel_48_min,logmel_48_max,logmel_49_mean,logmel_49_std,logmel_49_min,logmel_49_max,logmel_50_mean,logmel_50_std,logmel_50_min,logmel_50_max,logmel_51_mean,logmel_51_std,logmel_51_min,logmel_51_max,logmel_52_mean,logmel_52_std,logmel_52_min,logmel_52_max,logmel_53_mean,logmel_53_std,logmel_53_min,logmel_53_max,logmel_54_mean,logmel_54_std,logmel_54_min,logmel_54_max,logmel_55_mean,logmel_55_std,logmel_55_min,logmel_55_max,logmel_56_mean,logmel_56_std,logmel_56_min,logmel_56_max,logmel_57_mean,logmel_57_std,logmel_57_min,logmel_57_max,logmel_58_mean,logmel_58_std,logmel_58_min,logmel_58_max,logmel_59_mean,logmel_59_std,logmel_59_min,logmel_59_max,logmel_60_mean,logmel_60_std,logmel_60_min,logmel_60_max,logmel_61_mean,logmel_61_std,logmel_61_min,logmel_61_max,logmel_62_mean,logmel_62_std,logmel_62_min,logmel_62_max,logmel_63_mean,logmel_63_std,logmel_63_min,logmel_63_max,logmel_64_mean,logmel_64_std,logmel_64_min,logmel_64_max,logmel_delta_1_mean,logmel_delta_1_std,logmel_delta_1_min,logmel_delta_1_max,logmel_delta_2_mean,logmel_delta_2_std,logmel_delta_2_min,logmel_delta_2_max,logmel_delta_3_mean,logmel_delta_3_std,logmel_delta_3_min,logmel_delta_3_max,logmel_delta_4_mean,logmel_delta_4_std,logmel_delta_4_min,logmel_delta_4_max,logmel_delta_5_mean,logmel_delta_5_std,logmel_delta_5_min,logmel_delta_5_max,lo

In [22]:
train_ids = DF_TRAIN["Participant_ID"].unique()
dev_ids   = DF_DEV["Participant_ID"].unique()
test_ids  = DF_TEST["Participant_ID"].unique()

train_df = logmel_with_labels[logmel_with_labels["Participant_ID"].isin(train_ids)]
val_df   = logmel_with_labels[logmel_with_labels["Participant_ID"].isin(dev_ids)]
test_df  = logmel_with_labels[logmel_with_labels["Participant_ID"].isin(test_ids)]

print("Train:", train_df.shape, "Val:", val_df.shape, "Test:", test_df.shape)

Train: (107, 772) Val: (35, 772) Test: (47, 772)


In [23]:
exclude_cols = ["Participant_ID", "PHQ8_Score", "PHQ8_Binary", "Gender"]

feature_cols = [c for c in logmel_with_labels.columns if c not in exclude_cols]

X_train = train_df[feature_cols]
y_train = train_df["PHQ8_Score"]

X_val   = val_df[feature_cols]
y_val   = val_df["PHQ8_Score"]

X_test  = test_df[feature_cols]
y_test  = test_df["PHQ8_Score"]

print("Number of MFCC features:", len(feature_cols))

Number of MFCC features: 768


### Use Random Forest to Find the Most Important Features.

In [24]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

rf_for_fs = RandomForestRegressor(
    n_estimators=500,
    random_state=28,
    n_jobs=-1
)

rf_for_fs.fit(X_train, y_train)

importances = rf_for_fs.feature_importances_

In [44]:
top_k = 40  # or 50, or 30 — your choice

idx_sorted = np.argsort(importances)[::-1]  # descending
top_idx = idx_sorted[:top_k]

top_features = [feature_cols[i] for i in top_idx]

print("Top Logmel features (names):")
for f in top_features:
    print(f)

Top Logmel features (names):
logmel_43_std
logmel_delta2_5_min
logmel_delta_50_max
logmel_delta_27_std
logmel_delta_18_min
logmel_delta2_63_mean
logmel_delta2_26_std
logmel_7_mean
logmel_delta2_17_min
logmel_delta_39_min
logmel_delta_43_std
logmel_delta_4_std
logmel_delta_51_max
logmel_delta_23_mean
logmel_delta_49_max
logmel_delta2_2_min
logmel_delta2_25_max
logmel_delta_52_mean
logmel_42_std
logmel_delta_45_max
logmel_1_mean
logmel_delta2_49_std
logmel_delta_51_mean
logmel_delta_52_max
logmel_delta_27_min
logmel_delta_26_std
logmel_delta2_1_min
logmel_51_max
logmel_delta_60_mean
logmel_delta2_27_std
logmel_delta_24_std
logmel_delta_22_mean
logmel_delta2_4_std
logmel_delta2_7_min
logmel_delta2_11_max
logmel_delta_11_mean
logmel_delta_45_mean
logmel_52_max
logmel_2_mean
logmel_delta2_3_min


### Build reduced MFCC DataFrames using only selected features

In [33]:
# Reduced feature matrices
X_train_red = X_train[top_features].copy()
X_val_red   = X_val[top_features].copy()
X_test_red  = X_test[top_features].copy()

print("Reduced train shape:", X_train_red.shape)
print("Reduced val shape:", X_val_red.shape)
print("Reduced test shape:", X_test_red.shape)

Reduced train shape: (107, 40)
Reduced val shape: (35, 40)
Reduced test shape: (47, 40)


In [34]:
data_folder = root / "data"

logmel_reduced_all = logmel_with_labels[["Participant_ID", "PHQ8_Score"] + top_features]
logmel_reduced_all.to_csv(data_folder / "LIBROSA_LOGMEL_TOP40.csv", index=False)

print("Saved reduced MFCC feature set at:", data_folder / "LIBROSA_LOGMEL_TOP40.csv")

Saved reduced MFCC feature set at: C:\Users\DELL\Desktop\Conversational-Health-Analytics-\data\LIBROSA_LOGMEL_TOP40.csv


In [35]:
import pandas as pd
from pathlib import Path

data_path = Path(r"C:\Users\DELL\Desktop\Conversational-Health-Analytics-\data")

df_logmel_all = pd.read_csv(data_path / "LIBROSA_LOGMEL_TOP40.csv")
print(df_logmel_all.shape)
df_logmel_all.head()


(189, 42)


,Participant_ID,PHQ8_Score,logmel_43_std,logmel_delta2_5_min,logmel_delta_50_max,logmel_delta_27_std,logmel_delta_18_min,logmel_delta2_63_mean,logmel_delta2_26_std,logmel_7_mean,logmel_delta2_17_min,logmel_delta_39_min,logmel_delta_43_std,logmel_delta_4_std,logmel_delta_51_max,logmel_delta_23_mean,logmel_delta_49_max,logmel_delta2_2_min,logmel_delta2_25_max,logmel_delta_52_mean,logmel_42_std,logmel_delta_45_max,logmel_1_mean,logmel_delta2_49_std,logmel_delta_51_mean,logmel_delta_52_max,logmel_delta_27_min,logmel_delta_26_std,logmel_delta2_1_min,logmel_51_max,logmel_delta_60_mean,logmel_delta2_27_std,logmel_delta_24_std,logmel_delta_22_mean,logmel_delta2_4_std,logmel_delta2_7_min,logmel_delta2_11_max,logmel_delta_11_mean,logmel_delta_45_mean,logmel_52_max,logmel_2_mean,logmel_delta2_3_min
0,300.0,2,10.739304,-3.390566,6.035970,1.613482,-6.915091,9.477621e-05,0.937947,-42.335514,-4.551128,-7.116451,1.025413,1.774328,5.946761,-0.001723,6.007954,-2.975336,3.854547,-0.002546,10.849917,5.819147,-39.851742,0.592864,-0.002173,5.929493,-6.881640,1.656332,-1.843542,-21.620869,-0.001654,0.907104,1.588885,-0.002017,0.927267,-3.479960,5.761508,-0.004754,-0.001929,-21.745163,-36.666473,-3.895775
1,301.0,3,9.395409,-3.711049,5.515513,2.241750,-6.758570,-2.955596e-05,1.330216,-37.368904,-4.948042,-5.455584,1.697675,1.862585,5.318966,-0.000474,5.434238,-3.584485,4.427106,-0.000098,9.774688,5.579383,-34.786011,0.958078,-0.000067,4.828682,-6.730700,2.220926,-1.432690,-30.542015,0.000148,1.343100,2.221152,-0.000345,1.091842,-4.017005,4.647406,-0.000594,0.000253,-36.178806,-29.060829,-3.941930
2,302.0,4,5.783973,-3.374838,4.380237,1.681334,-5.304066,-7.131106e-04,1.019084,-32.149399,-3.263655,-4.337917,1.031796,1.442157,4.331590,-0.003016,4.877388,-2.740170,3.888376,-0.001019,6.057352,4.227523,-26.297705,0.635646,-0.001046,4.160639,-5.123510,1.686172,-1.945076,-33.668652,-0.000683,1.037769,1.753835,-0.004311,0.878476,-3.209999,3.398974,-0.002828,-0.001782,-34.625015,-20.675009,-3.446138
3,303.0,0,8.490192,-3.621092,5.452006,2.105390,-7.104780,2.284568e-04,1.305185,-39.742962,-4.837091,-6.417822,1.554623,1.787458,5.292192,-0.001742,5.712991,-2.230157,4.562909,-0.000424,8.184835,6.293627,-37.777328,0.779309,-0.000493,5.600675,-7.099382,2.164683,-2.539845,-32.798985,-0.000130,1.288230,2.169303,-0.001437,0.972501,-3.872371,4.577280,-0.001383,-0.000674,-32.301853,-44.292381,-3.121050
4,304.0,6,7.878151,-3.339752,4.832123,1.909255,-7.139767,2.419532e-10,1.134411,-47.634342,-5.684505,-5.698740,1.347867,1.569708,4.635792,0.000206,6.025497,-3.549773,4.249362,0.000207,7.976190,5.967069,-41.200623,0.699520,0.000530,4.726497,-6.308327,1.950240,-3.170179,-40.946724,0.000392,1.104353,1.936945,0.000470,0.846326,-4.459381,3.868308,0.000127,0.000380,-42.157715,-47.938351,-3.846102


### Feature Engineering.

### Handle Missing, Infinite, and Constant Features

In [49]:
import numpy as np

# Replace inf with NaN
X_train_red.replace([np.inf, -np.inf], np.nan, inplace=True)
X_val_red.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test_red.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN with column mean
X_train_red = X_train_red.fillna(X_train_red.mean())
X_val_red   = X_val_red.fillna(X_train_red.mean())
X_test_red  = X_test_red.fillna(X_train_red.mean())

# Remove constant columns
constant_cols = X_train_red.columns[X_train_red.nunique() == 1]
print("Removed constant features:", constant_cols.tolist())

X_train_red.drop(columns=constant_cols, inplace=True)
X_val_red.drop(columns=constant_cols, inplace=True)
X_test_red.drop(columns=constant_cols, inplace=True)


Removed constant features: []


### Outlier Detection & Removal (MFCC ranges vary widely).


In [50]:
Q1 = X_train_red.quantile(0.25)
Q3 = X_train_red.quantile(0.75)
IQR = Q3 - Q1

# Keep only non-outliers
mask = ~((X_train_red < (Q1 - 1.5 * IQR)) | (X_train_red > (Q3 + 1.5 * IQR))).any(axis=1)

X_train_red = X_train_red[mask]
y_train = y_train[mask]

### Feature Scaling.

In [51]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_red)
X_val_scaled   = scaler.transform(X_val_red)
X_test_scaled  = scaler.transform(X_test_red)


### Train and Evaluate the Three Best Regression Models.

### Evaluation Function

In [52]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_model(model, X, y, name="Model"):
    preds = model.predict(X)
    mae  = mean_absolute_error(y, preds)
    mse  = mean_squared_error(y, preds)
    rmse = np.sqrt(mse)
    r2   = r2_score(y, preds)

    print(f"\n{name} Validation Performance")
    print("---------------------------------------")
    print("MAE :", mae)
    print("MSE :", mse)
    print("RMSE:", rmse)
    print("R²  :", r2)

    return [mae, mse, rmse, r2]

### Train XGBoost (Best Model for AVEC-type datasets).

In [53]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=600,
    learning_rate=0.03,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    reg_alpha=0.0,
    random_state=28
)

xgb.fit(X_train_scaled, y_train)
xgb_results = evaluate_model(xgb, X_val_scaled, y_val, "XGBoost")


XGBoost Validation Performance
---------------------------------------
MAE : 6.062353610992432
MSE : 59.09469223022461
RMSE: 7.687307215808707
R²  : -0.4007546901702881


### Train LightGBM (Fast and Excellent for MFCC features).

In [54]:
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(
    n_estimators=800,
    learning_rate=0.03,
    num_leaves=50,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=28
)

lgb.fit(X_train_scaled, y_train)
lgb_results = evaluate_model(lgb, X_val_scaled, y_val, "LightGBM")

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 28, number of used features: 0
[LightGBM] [Info] Start training from score 5.607143
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

c:\Users\DELL\anaconda3\envs\tf_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


### Train Random Forest.

In [55]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=600,
    random_state=28
)

rf.fit(X_train_red, y_train)    # no scaling needed
rf_results = evaluate_model(rf, X_val_red, y_val, "Random Forest")


Random Forest Validation Performance
---------------------------------------
MAE : 5.8020476190476185
MSE : 49.94087738095238
RMSE: 7.066885974809016
R²  : -0.18377660200593415
